In [8]:
import pandas as pd

# Load the tree data
df = pd.read_csv("../../datasets/OBARD_filtered.csv")

# Define valid life stages
valid_life_stages = ['young', 'early_mature', 'semi_mature', 'mature', 'late_mature']

# Filter valid life stages
df = df[df['life_stage'].isin(valid_life_stages)]

# Convert columns to numeric
df['height_m'] = pd.to_numeric(df['height_m'], errors='coerce')
df['crown_radius_minimum_bounding_circle'] = pd.to_numeric(
    df['crown_radius_minimum_bounding_circle'], errors='coerce'
)

# Drop rows with missing values in required columns
df = df.dropna(subset=['height_m', 'crown_radius_minimum_bounding_circle', 'latin_name_clean'])

# Group by life_stage and latin_name_clean
summary = df.groupby(['life_stage', 'latin_name_clean']).agg({
    'height_m': 'mean',
    'crown_radius_minimum_bounding_circle': 'mean'
}).reset_index()

# Round results to 1 decimal
summary['average_height_m'] = summary['height_m'].round(1)
summary['minimum_crown_radius_m'] = summary['crown_radius_minimum_bounding_circle'].round(1)

# Drop intermediate columns
summary = summary.drop(columns=['height_m', 'crown_radius_minimum_bounding_circle'])

# Save to CSV
summary.to_csv("life_stage_tree_summary.csv", index=False)

print("Saved life_stage_tree_summary.csv")

Saved life_stage_tree_summary.csv


In [10]:
import pandas as pd
import json

# Load the CSV
df = pd.read_csv("life_stage_tree_summary.csv")

# Define crown height percentage (e.g. 70%)
crown_percentage = 0.3

# Build a flat list of entries
entries = []

for _, row in df.iterrows():
    tree_ht = round(row['average_height_m'], 1)
    crown_dia = round(2 * row['minimum_crown_radius_m'], 1)
    crown_ht = round(crown_percentage * tree_ht, 1)

    entry = {
        "latin name": row['latin_name_clean'],
        "age": row['life_stage'],
        "crown dia": crown_dia,
        "tree ht": tree_ht,
        "trunk ht": crown_ht
    }

    entries.append(entry)

# Save to JSON
with open("obard_trees.json", "w") as f:
    json.dump(entries, f, indent=2)

print("Saved tree_life_stage_flat.json")

Saved tree_life_stage_flat.json
